# Text Data

## Pre-introduction

We'll be spending a lot of time today manipulating text. Make sure you remember how to split, join, and search strings.

## Introduction

We've spent a lot of time in python dealing with text data, and that's because text data is everywhere. It is the primary form of communication between persons and persons, persons and computers, and computers and computers. The kind of inferential methods that we apply to text data, however, are different from those applied to tabular data. 

This is partly because documents are typically specified in a way that expresses both structure and content using text (i.e. the document object model).

Largely, however, it's because text is difficult to turn into numbers in a way that preserves the information in the document. Today, we'll talk about dominant language model in NLP and the basics of how to implement it in Python.

### The term-document model

This is also sometimes referred to as "bag-of-words" by those who don't think very highly of it. The term document model looks at language as individual communicative efforts that contain one or more tokens. The kind and number of the tokens in a document tells you something about what is attempting to be communicated, and the order of those tokens is ignored.

To start with, let's load a document.

In [2]:
import nltk
#nltk.download('webtext')
document = nltk.corpus.webtext.open('grail.txt').read()

[nltk_data] Downloading package webtext to /Users/dillon/nltk_data...
[nltk_data]   Package webtext is already up-to-date!


/Users/dillon/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


Let's see what's in this document

In [3]:
len(document.split('\n'))

1192

In [4]:
document.split('\n')[0:10]

['SCENE 1: [wind] [clop clop clop] ',
 'KING ARTHUR: Whoa there!  [clop clop clop] ',
 'SOLDIER #1: Halt!  Who goes there?',
 'ARTHUR: It is I, Arthur, son of Uther Pendragon, from the castle of Camelot.  King of the Britons, defeator of the Saxons, sovereign of all England!',
 'SOLDIER #1: Pull the other one!',
 'ARTHUR: I am, ...  and this is my trusty servant Patsy.  We have ridden the length and breadth of the land in search of knights who will join me in my court at Camelot.  I must speak with your lord and master.',
 'SOLDIER #1: What?  Ridden on a horse?',
 'ARTHUR: Yes!',
 "SOLDIER #1: You're using coconuts!",
 'ARTHUR: What?']

It looks like we've gotten ourselves a bit of the script from Monty Python and the Holy Grail. Note that when we are looking at the text, part of the structure of the document is written in tokens. For example, stage directions have been placed in brackets, and the names of the person speaking are in all caps.

## Regular expressions

If we wanted to read out all of the stage directions for analysis, or just King Arthur's lines, doing so in base python string processing will be very difficult. Instead, we are going to use regular expressions. Regular expressions are a method for string manipulation that match patterns instead of bytes.

In [14]:
import re
snippet = "I fart in your general direction! Your mother was a hamster, and your father smelt of elderberries!"
re.search(r'mother', snippet)

<_sre.SRE_Match object; span=(39, 45), match='mother'>

Just like with `str.find`, we can search for plain text. But `re` also gives us the option for searching for patterns of bytes - like only alphabetic characters.

In [17]:
re.search(r'[a-z]', snippet)

<_sre.SRE_Match object; span=(2, 3), match='f'>

In this case, we've told re to search for the first sequence of bytes that is only composed of lowercase letters between `a` and `z`. We could get the letters at the end of each sentence by including a bang at the end of the pattern.

In [18]:
re.search(r'[a-z]!', snippet)

<_sre.SRE_Match object; span=(31, 33), match='n!'>

If we wanted to pull out just the stage directions from the screenplay, we might try a pattern like this:

In [8]:
re.findall(r'[a-zA-Z]', document)[0:10]

['S', 'C', 'E', 'N', 'E', 'w', 'i', 'n', 'd', 'c']

So that's obviously no good. There are two things happening here:

1. `[` and `]` do not mean 'bracket'; they are special characters which mean 'any thing of this class'
2. we've only matched one letter each

A better regular expression, then, would wrap this in escaped brackets, and include a command saying more than one letter.

Re is flexible about how you specify numbers - you can match none, some, a range, or all repetitions of a sequence or character class.

character | meaning
----------|--------
`{x}`     | exactly x repetitions
`{x,y}`   | between x and y repetitions
`?`       | 0 or 1 repetition
`*`       | 0 or many repetitions
`+`       | 1 or many repetitions

In [9]:
re.findall(r'\[[a-zA-Z]+\]', document)[0:10]

['[wind]',
 '[thud]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]']

This is better, but it's missing that `[clop clop clop]` we saw above. This is because we told the regex engine to match any alphabetic character, but we did not specify whitespaces, commas, etc. to match these, we'll use the dot operator, which will match anything expect a newline.

Part of the power of regular expressions are their special characters. Common ones that you'll see are:

character | meaning
----------|--------
`.`       | match anything except a newline
`^`       | match the start of a line
`$`       | match the end of a line
`\s`      | matches any whitespace or newline

Finally, we need to fix this `+` character. It is a 'greedy' operator, which means it will match as much of the string as possible. To see why this is a problem, try:

In [11]:
snippet = 'This is [cough cough] and example of a [really] greedy operator'
re.findall(r'\[.+\]', snippet)

['[cough cough] and example of a [really]']

Since the operator is greedy, it is matching everything inbetween the first open and the last close bracket. To make `+` consume the least possible amount of string, we'll add a `?`.

In [12]:
p = re.compile(r'\[.+?\]')
re.findall(p, document)[0:10]

['[wind]',
 '[clop clop clop]',
 '[clop clop clop]',
 '[clop clop clop]',
 '[thud]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]',
 '[clang]']

What if we wanted to grab all of Arthur's speech? This one is a little trickier, since:

1. It is not conveniently bracketed; and,
2. We want to match on ARTHUR, but not to capture it

If we wanted to do this using base string manipulation, we would need to do something like:

```
split the document into lines
create a new list of just lines that start with ARTHUR
create a newer list with ARTHUR removed from the front of each element
```

Regex gives us a way of doing this in one line, by using something called groups. Groups are pieces of a pattern that can be ignored, negated, or given names for later retrieval.

character | meaning
----------|--------
`(x)`     | match x
`(?:x)`   | match x but don't capture it
`(?P<x>)` | match something and give it name x
`(?=x)`   | match only if string is followed by x
`(?!x)`   | match only if string is not followed by x

In [19]:
p = re.compile(r'(?:ARTHUR: )(.+)')
re.findall(p, document)[0:10]

['Whoa there!  [clop clop clop] ',
 'It is I, Arthur, son of Uther Pendragon, from the castle of Camelot.  King of the Britons, defeator of the Saxons, sovereign of all England!',
 'I am, ...  and this is my trusty servant Patsy.  We have ridden the length and breadth of the land in search of knights who will join me in my court at Camelot.  I must speak with your lord and master.',
 'Yes!',
 'What?',
 'So?  We have ridden since the snows of winter covered this land, through the kingdom of Mercea, through--',
 'We found them.',
 'What do you mean?',
 'The swallow may fly south with the sun or the house martin or the plover may seek warmer climes in winter, yet these are not strangers to our land?',
 'Not at all.  They could be carried.']

Because we are using `findall`, the regex engine is capturing and returning the normal groups, but not the non-capturing group. For complicated, multi-piece regular expressions, you may need to pull groups out separately. You can do this with names.

In [22]:
p = re.compile(r'(?P<name>[A-Z ]+)(?::)(?P<line>.+)')
match = re.search(p, document)
match

<_sre.SRE_Match object; span=(34, 77), match='KING ARTHUR: Whoa there!  [clop clop clop] '>

In [23]:
match.group('name'), match.group('line')

('KING ARTHUR', ' Whoa there!  [clop clop clop] ')

#### Your turn!

Go into the challenges folder and try your hand at challenge A.

## Tokenizing

Let's grab Arthur's speech from above, and see what we can learn about Arthur from it.

In [25]:
p = re.compile(r'(?:ARTHUR: )(.+)')
arthur = ' '.join(re.findall(p, document))
arthur[0:100]

'Whoa there!  [clop clop clop]  It is I, Arthur, son of Uther Pendragon, from the castle of Camelot. '

In our model for natural language, we're interested in words. The document is currently a continuous string of bytes, which isn't ideal. You might be tempted to separate this into words using your newfound regex knowledge:

In [26]:
p = re.compile(r'\w+', flags=re.I)
re.findall(p, arthur)[0:10]

['Whoa', 'there', 'clop', 'clop', 'clop', 'It', 'is', 'I', 'Arthur', 'son']

But this is problematic for languages that make extensive use of punctuation. For example, see what happens with:

In [27]:
re.findall(p, "It isn't Dav's cheesecake that I'm worried about")

['It',
 'isn',
 't',
 'Dav',
 's',
 'cheesecake',
 'that',
 'I',
 'm',
 'worried',
 'about']

The practice of pulling apart a continuous string into units is called "tokenizing", and it creates "tokens". NLTK, the canonical library for NLP in Python, has a couple of implementations for tokenizing a string into words.

In [33]:
from nltk import word_tokenize
word_tokenize("It isn't Dav's cheesecake that I'm worried about")

['It',
 'is',
 "n't",
 'Dav',
 "'s",
 'cheesecake',
 'that',
 'I',
 "'m",
 'worried',
 'about']

The distinction here is subtle, but look at what happened to "isn't". It's been separated into "IS" and "N'T", which is more in keeping with the way contractions work in English.

In [ ]:
tokens = word_tokenize(arthur)
tokens[0:10]